In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_pickle('../data/data_step1.pkl')
data.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [3]:
data['Item'].unique()

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'UNKNOWN',
       'Sandwich', nan, 'ERROR', 'Juice', 'Tea'], dtype=object)

In [3]:
data['Payment Method'].unique()

array(['Credit Card', 'Cash', 'UNKNOWN', 'Digital Wallet', 'ERROR', nan],
      dtype=object)

In [4]:
data['Location'].unique()

array(['Takeaway', 'In-store', 'UNKNOWN', nan, 'ERROR'], dtype=object)

In [5]:
# Item = NaN
len(data.loc[data['Item'].isna(), 'Item'])

333

In [4]:
# Item = UNKNOWN ou ERROR
data.loc[(data['Item'] == 'UNKNOWN') | (data['Item'] == 'ERROR'), 'Item'].count()

np.int64(636)

In [7]:
# Payment Method = NaN
len(data.loc[data['Payment Method'].isna(), 'Payment Method'])

2579

In [8]:
# Payment Method = UNKNOWN ou ERROR
data.loc[(data['Payment Method'] == 'UNKNOWN') | (data['Payment Method'] == 'ERROR'), 'Payment Method'].count()

np.int64(599)

Optou-se por analisar quais os possíveis Itens que foram preenchidos como NaN, "ERROR" e UNKNOWN".

Explicação do codigo a seguir: Agrupamento de dados por Item com retorno da coluna Price Per Unit, contagem de ocorrências do preço em cada Item e organização em colunas com "unstack".

In [5]:
# Verificação de Price Per Unit em relação a cada item (numero de ocorrências)
data.groupby('Item')['Price Per Unit'].value_counts().unstack().fillna(0) 

Price Per Unit,1.0,1.5,2.0,3.0,4.0,5.0
Item,,,,,,
Cake,0.0,0.0,0.0,1139.0,0.0,0.0
Coffee,0.0,0.0,1165.0,0.0,0.0,0.0
Cookie,1092.0,0.0,0.0,0.0,0.0,0.0
ERROR,34.0,37.0,31.0,77.0,61.0,39.0
Juice,0.0,0.0,0.0,1171.0,0.0,0.0
Salad,0.0,0.0,0.0,0.0,0.0,1148.0
Sandwich,0.0,0.0,0.0,0.0,1131.0,0.0
Smoothie,0.0,0.0,0.0,0.0,1096.0,0.0
Tea,0.0,1089.0,0.0,0.0,0.0,0.0


A função seguinte permite descobrir com que frequência cada combinação de Item, Quantity e Price Per Unit aparece.

In [11]:
# Verificando a moda de Quantity em cada Item
item_group = (
    data[~data['Item'].isin(['UNKNOWN', 'ERROR']) & data['Item'].notna()] # Apenas valores não nulos e sem erro
    .groupby(['Quantity', 'Price Per Unit', 'Item'])
    .size() #Aqui é onde aparece a contagem de quantas vezes a combinação apareceu
    .reset_index(name='count') # Coluna count no dataframe (onde armazena o count)
    .sort_values('count',ascending=False) # Ordena da combinação mais frequente até a menos frequente
)
item_group.head()

,Quantity,Price Per Unit,Item,count
34,5.0,2.0,Coffee,258
35,5.0,3.0,Cake,255
37,5.0,4.0,Sandwich,254
39,5.0,5.0,Salad,248
20,3.0,3.0,Juice,244


Agora, fazemos o preenchimento dos Itens com base no Price Per Unit e Quantity:

Passo 1: Criar uma cópia dos dados para aplicar a máscara (item_group);

Passo 2: Criar a máscara com os dados desconhecidos;

Passo 3: Iterar sobre a cópia dos dados para aplicar as mudanças.

In [7]:
# Copia dos dados
data_copy = data.copy()

# Criação da mascara com a copia
mask = (data_copy['Item'].isin(['UNKNOWN', 'ERROR']) | (data['Item'].isna()))

In [8]:
# Iterando sobre a copia dos dados com base na mask
for i, row in data_copy[mask].iterrows():
    price = row['Price Per Unit']
    quant = row['Quantity']

   # Procura a combinação no df
    match = item_group[
    np.isclose(item_group['Price Per Unit'], price, rtol=1e-03) &
    (item_group['Quantity'] == quant) 
    ]

    # Aplicando o match se encontrar correspondência
    if not match.empty:
        data_copy.at[i, 'Item'] = match.iloc[0]['Item']

In [9]:
data_copy['Item'].value_counts(dropna=False)

Item
Juice       1359
Sandwich    1301
Coffee      1284
Salad       1270
Cookie      1209
Tea         1199
Cake        1185
Smoothie    1139
NaN           23
UNKNOWN       18
ERROR         13
Name: count, dtype: int64

In [10]:
data_copy.to_pickle('../data/data_step2.pkl')